In [4]:
##Estimación de rezago habitacional

#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

In [5]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [6]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_viviendas_csv.zip"

In [7]:
years<-c("2016", "2018", "2020", "2022")

In [12]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)
  
  # Descargar el archivo
  download.file(url, destfile = destfile)
  
  # Descomprimir el archivo
  unzip(destfile, exdir = "data")
  
  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/viviendas.csv", 
    to = paste0("data/viviendas_", years[i], ".csv")
  )
}    

### Función de estimación

In [62]:
estimacion <- function(path, variable) {
  # Cargar los datos
  datos <- read_csv(path)
  
  datos <- datos %>%
    mutate(
      across(starts_with("mat") & where(is.character),
             ~ replace_na(parse_number(.x, na = c('', 'NA', '&')), 0)),
      rezago = if_else(
        ((tot_resid / num_cuarto) > 2.5) |
          (mat_pared %in% 1:6) |
          (mat_techos %in% c(1:4, 6, 7, 9)) |
          (mat_pisos == 1) |
          (excusado == 2),
        "En rezago",
        "Fuera de rezago"
      )
    ) %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")
  
  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)
  
  # Estimar rezago habitacional
  rezago <- mydesign %>%
    group_by(rezago) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    mutate(pct = round(pct * 100, 2)) %>%
    filter(rezago == "En rezago") %>%
    # Añadir año
    mutate(year = substr(path, 16, 19))
  
  
  
  # Estimar viviendas sin escrituras
  escrituras <- mydesign %>%
    filter(tenencia %in% c(3,4)) %>%
group_by(escrituras) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    filter(escrituras == 3) %>%
    mutate(pct = round(pct * 100, 2),
           escrituras = case_when(
        escrituras == 3 ~ "Sin escrituras"
))%>%
    
mutate(year = substr(path, 16, 19))
  
  # Devolver el resultado basado en la variable
  if (variable == "rezago") {
    return(rezago)
  } else if (variable == "sin_escrituras") {
    return(escrituras)
  } else {
    stop("Variable no reconocida. Use 'rezago' o 'sin_escrituras'.")
  }
}


In [63]:
#Construir cuadros de rezago habitacional

tabla_rezago<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
  estimacion,
  variable = "rezago"
) %>%
  select(year, viviendas, pct)

tabla_rezago


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

year,viviendas,pct
<chr>,<dbl>,<dbl>
2016,272200,9.81
2018,218501,8.02
2020,198021,7.31
2022,175242,5.94


In [64]:
#Construir cuadros de viviendas sin escrituras
#Viviendas sin escrituras corresponden a viviendas propias y propias que se están pagando y no cuentan con escrituras

tabla_escrituras<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable = "sin_escrituras"
    ) %>%
    select(year, viviendas, pct)
    
tabla_escrituras

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

year,viviendas,pct
<chr>,<dbl>,<dbl>
2016,266547,17.71
2018,246283,17.22
2020,259056,16.23
2022,310191,17.63
